In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from random import shuffle
import numpy as np
from nltk.corpus import stopwords
import itertools
import nltk
import numpy as np
from keras.utils.np_utils import to_categorical
from numpy import array
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives
from keras.layers import Input, RepeatVector, LSTM, Embedding
from keras.models import Model
from keras.backend import permute_dimensions, gather
from numpy import array
from keras.preprocessing.sequence import pad_sequences
import random
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from time import time
from keras.preprocessing.text import Tokenizer

In [ ]:
length = 100

original_data = []
paraphrase_data = []

with open('source.txt', 'r', encoding = 'utf8') as f:
    for line in f:
        original_data.append(line.strip())


with open('target.txt', 'r', encoding = 'utf8') as f:
    for line in f:
        paraphrase_data.append(line.strip())
        
paraphrase_data = paraphrase_data[:length]
original_data = original_data[:length]
print (len(original_data))
print (len(paraphrase_data))
print (original_data)
print (paraphrase_data)


In [ ]:
vocab = []
for word in original_data:
    if word not in vocab:
        vocab.append(word)

for word in paraphrase_data:
    if word not in vocab:
        vocab.append(word)
        
# print (len(vocab))  
# print (vocab)
vocab_index = {}
count = 1
for word in vocab:
    vocab_index[word] = count
    count += 1
    
vocab_index['EOS'] = 0   
# print (vocab_index)
vocab_size = len(vocab_index)  
print (vocab_size)

int_to_vocab = {}
for character, value in vocab_index.items():
    int_to_vocab[value] = character

# print (int_to_vocab)

orig_data = []
for word in original_data:
    orig_data.append([vocab_index[word],vocab_index['EOS']])
    
para_data = []
for word in paraphrase_data:
    para_data.append([vocab_index[word],vocab_index['EOS']])    

    
def manual_one_hot(lines):
    res = []
    for line in lines:
        res.append(to_categorical(line, num_classes=vocab_size))
    return array(res)    
# print (orig_data)
# print (para_data)    

para_data = np.array(para_data)  
orig_data = np.array(orig_data)

target_paraphrase = manual_one_hot(para_data)


print (orig_data[0])   
print (para_data[0])
# print (len(orig_data[0]))
# print (orig_data)
print (orig_data.shape)
# print (len(para_data[0]))
print (para_data.shape)
print (target_paraphrase.shape)

In [ ]:
embeddings_index = dict()
f = open('glove.6B.50d.txt', encoding = 'utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
print (vocab_size)
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in vocab_index.items():
#     print (word, i)
    embedding_vector = embeddings_index.get(word)
#     print (embedding_vector, word, i)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        print (word, i)
        
print (embedding_matrix.shape)   
print (embeddings_index['dealt'] - embedding_matrix[3])

In [ ]:
np.set_printoptions(threshold=np.nan)
input_dim = len(vocab_index)
time_steps = orig_data.shape[1]
batch_size = 1
print("time_steps:" + str(time_steps))
print("input_dim:" + str(input_dim))
interdim = 128
latentdim = 128

In [ ]:
def create_lstm_vae(input_dim,
                    timesteps,
                    batch_size,
                    intermediate_dim,
                    latent_dim,
                    epsilon_std=1., ):
    """
    Creates an LSTM Variational Autoencoder (VAE). Returns VAE, Encoder, Generator. 

    # Arguments
        input_dim: int.
        timesteps: int, input timestep dimension.
        batch_size: int.
        intermediate_dim: int, output shape of LSTM. 
        latent_dim: int, latent z-layer shape. 
        epsilon_std: float, z-layer sigma.


    # References
        - [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
        - [Generating sentences from a continuous space](https://arxiv.org/abs/1511.06349)
    """

    def crop_dimension():
        def func(x):
            x = K.permute_dimensions(x, (1, 0, 2))
            x = K.gather(x, [i for i in range(timesteps)])
            x = K.permute_dimensions(x, (1, 0, 2))
            return x

        return Lambda(func)

    def crop(dimension, start, end):
        # Crops (or slices) a Tensor on a given dimension from start to end
        # example : to crop tensor x[:, :, 5:10]
        # call slice(2, 5, 10) as you want to crop on the second dimension
        def func(x):
            if dimension == 0:
                return x[start: end]
            if dimension == 1:
                return x[:, start: end]
            if dimension == 2:
                return x[:, :, start: end]
            if dimension == 3:
                return x[:, :, :, start: end]
            if dimension == 4:
                return x[:, :, :, :, start: end]

        return Lambda(func)

    crop_layer = crop_dimension()
    # drop_out_layer = Dropout(rate=0.0)
    
                                    # original sentence encoding
        
    original_sentence_input = Input(shape=(None,), name="OriginalInput_1")
    # original_sentence_input = Input(shape=(timesteps, input_dim,), name="OriginalInput_1")
    
    original_encoder_layer_1 = LSTM(intermediate_dim, return_sequences=True, name="OriginalEncoderLSTM_1")
    original_encoder_layer_2 = LSTM(intermediate_dim, return_sequences=True, name="OriginalEncoderLSTM_2")
    original_encoder_layer_3 = LSTM(intermediate_dim, return_sequences=True, name="OriginalEncoderLSTM_3")

    embedding_layer = Embedding(198,50, weights = [embedding_matrix], mask_zero=True, trainable = False)

    encoded_original = embedding_layer(original_sentence_input)
    encoded_original = original_encoder_layer_1(encoded_original)
    # encoded_original = drop_out_layer(encoded_original)
    encoded_original = original_encoder_layer_2(encoded_original)
    # encoded_original = drop_out_layer(encoded_original)
    encoded_original = original_encoder_layer_3(encoded_original)

    # paraphrase sentence encoder
    paraphrase_sentence_encoder_layer1 = LSTM(intermediate_dim, return_sequences=True, name="ParaphraseEncoderLSTM_1")
    paraphrase_sentence_encoder_layer2 = LSTM(intermediate_dim, return_sequences=True, name="ParaphraseEncoderLSTM_2")
    paraphrase_sentence_encoder_layer3 = LSTM(intermediate_dim, return_sequences=True, name="ParaphraseEncoderLSTM_3")

    # x = Input(shape=(timesteps, input_dim,), name="ParaphraseInput_1")
    x = Input(shape=(None,), name="ParaphraseInput_1")
    embedded_x = embedding_layer(x)
    # LSTM encoding
    # h = LSTM(intermediate_dim)(x)
    
                                    # merge original and paraphrase
        
    encoded_original = crop_layer(encoded_original)
    embedded_x = crop_layer(embedded_x)
    
    h = keras.layers.concatenate([encoded_original, embedded_x], axis=-1)
    h = paraphrase_sentence_encoder_layer1(h)
    # h = drop_out_layer(h)
    h = paraphrase_sentence_encoder_layer2(h)
    # h = drop_out_layer(h)
    h = paraphrase_sentence_encoder_layer3(h)

    # VAE Z layer
    # h = Dense(intermediate_dim)(h)
    # h = LSTM(intermediate_dim, return_sequences=True)(h)
    # h = RepeatVector(timesteps)(h)
    # h = LSTM(intermediate_dim)(h)

    # test begin
    # z_mean = Dense(latent_dim)(h)
    # z_log_sigma = Dense(latent_dim)(h)
    # test ends
    z_mean = LSTM(latent_dim)(h)
    z_log_sigma = LSTM(latent_dim)(h)

    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        # return z_mean + z_log_sigma * epsilon
        return z_mean + K.exp(z_log_sigma) * epsilon
        # return z_mean + K.exp(z_log_sigma / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

                                    # decoded LSTM layer
        
    # decoder_h = LSTM(timesteps, return_sequences=True, name="DecoderH_1")
    # decoder_mean = LSTM(input_dim, return_sequences=True, name="DecoderMean_1")
    decoder_h = Dense(intermediate_dim, name="DecoderH_1", activation="relu")
    decoder_mean = Dense(input_dim, name="DecoderMean_1", activation="sigmoid")

    # paraphrase sentence decoder
    paraphrase_sentence_decoder_layer1 = LSTM(intermediate_dim, return_sequences=True,
                                              name="ParaphraseSentenceDecoder_1")
    paraphrase_sentence_decoder_layer2 = LSTM(intermediate_dim, return_sequences=True,
                                              name="ParaphraseSentenceDecoder_2")
    paraphrase_sentence_decoder_layer3 = LSTM(intermediate_dim, return_sequences=True,
                                              name="ParaphraseSentenceDecoder_3")

    h_decoded = RepeatVector(timesteps)(z)

    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer1(encoded_original)

    Lambda(lambda x: K.permute_dimensions(x, (1, 0, 2)))
    paraphrase_sentence_decoded = keras.layers.concatenate([h_decoded, paraphrase_sentence_decoded],
                                                           axis=-1)
    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer2(paraphrase_sentence_decoded)
    paraphrase_sentence_decoded = keras.layers.concatenate([h_decoded, paraphrase_sentence_decoded],
                                                           axis=-1)
    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer3(paraphrase_sentence_decoded)

    h_decoded = decoder_h(paraphrase_sentence_decoded)

                                    # decoded layer
    x_decoded_mean = decoder_mean(h_decoded)

                                # end-to-end autoencoder
    vae = Model([original_sentence_input, x], x_decoded_mean)

                            # encoder, from inputs to latent space
    encoder = Model([original_sentence_input, x], z_mean)

    # generator, from latent space to reconstructed inputs
    decoder_latent_input = Input(shape=(latent_dim,), name="DecoderLatentInput_1")
    decoder_input_repeated = RepeatVector(timesteps)(decoder_latent_input)
    # decoder original encoder
    # decoder_original_input = Input(shape=(timesteps, input_dim,), name="DecoderOriginalInput_1")
    decoder_original_input = Input(shape=(None,), name="DecoderOriginalInput_1")
    decoder_original_encoded = embedding_layer(decoder_original_input)
    decoder_original_encoded = original_encoder_layer_1(decoder_original_encoded)
    # decoder_original_encoded = drop_out_layer(decoder_original_encoded)
    decoder_original_encoded = original_encoder_layer_2(decoder_original_encoded)
    # decoder_original_encoded = drop_out_layer(decoder_original_encoded)
    decoder_original_encoded = original_encoder_layer_3(decoder_original_encoded)

    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer1(decoder_original_encoded)
    paraphrase_sentence_decoded = crop_layer(paraphrase_sentence_decoded)
    paraphrase_sentence_decoded = keras.layers.concatenate([decoder_input_repeated, paraphrase_sentence_decoded],
                                                           axis=-1)
    # paraphrase_sentence_decoded = drop_out_layer(paraphrase_sentence_decoded)
    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer2(paraphrase_sentence_decoded)
    paraphrase_sentence_decoded = keras.layers.concatenate([decoder_input_repeated, paraphrase_sentence_decoded],
                                                           axis=-1)
    # paraphrase_sentence_decoded = drop_out_layer(paraphrase_sentence_decoded)
    paraphrase_sentence_decoded = paraphrase_sentence_decoder_layer3(paraphrase_sentence_decoded)

    _h_decoded = decoder_h(paraphrase_sentence_decoded)
    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = Model([decoder_latent_input, decoder_original_input], _x_decoded_mean)

    # def vae_loss(x, x_decoded_mean):
    #     xent_loss = objectives.mse(x, x_decoded_mean)
    #     kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
    #     loss = xent_loss + kl_loss
    #     return loss
    def vae_loss(x, x_decoded_mean):
        xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
        return xent_loss + kl_loss

    # def vae_loss(x, x_decoded_mean):
    #     xent_loss = input_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    #     kl_loss = - 0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    #     return K.mean(xent_loss + kl_loss)
    # def vae_loss(x, x_decoded_mean):
    #     # E[log P(X|z)]
    #     recon = K.sum(K.binary_crossentropy(x, x_decoded_mean), axis=1)
    #     # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    #     kl = 0.5 * K.sum(K.exp(z_log_sigma) + K.square(z_mean) - 1. - z_log_sigma, axis=1)
    #     return recon + kl

    # sgd = SGD(lr=0.00005, decay=1e-6, momentum=0.9, nesterov=True)
    #sgd = SGD(lr=0.00005)
    # rmsprop = RMSprop()
    vae.compile(optimizer=Adam(lr = 0.00001), loss=vae_loss, metrics = ['accuracy'])

    return vae, encoder, generator


In [ ]:
vae, enc, gen = create_lstm_vae(input_dim, timesteps=time_steps, batch_size=batch_size,
                                intermediate_dim=interdim, latent_dim=latentdim, epsilon_std=1.,)

In [ ]:
vae.load_weights("vae_model.h5")
enc.load_weights("enc.h5")
gen.load_weights("gen.h5") 

history = vae.fit([orig_data, para_data], target_paraphrase, epochs=10, batch_size=batch_size, shuffle=True, validation_split=0.2,
#                     validation_data=([test_original, test_paraphrase], test_paraphrase),
#                     callbacks=[TensorBoard(log_dir="logs_ppdb/{}".format(time()))]
                 )
 
vae.save_weights("vae_model.h5")
enc.save_weights("enc.h5")
gen.save_weights("gen.h5") 

In [ ]:
for t in range(10):
    t = random.randint(0,99)
    sent_encoded = enc.predict([orig_data[t:t + 1], para_data[t:t + 1]], batch_size=batch_size)  
    sent_decoded = gen.predict([sent_encoded, orig_data[t:t + 1]], batch_size=batch_size)
    print ('prediction', end = ' ')
    for n in sent_decoded[0]:
        if int_to_vocab[np.argmax(n)] != '<PAD>' and int_to_vocab[np.argmax(n)] != 'EOS': 
            print(int_to_vocab[np.argmax(n)], end=" ")    
    print ('\noriginal ', original_data[t])
    print ('paraphrase ', paraphrase_data[t])
    print ('\n')

In [ ]:
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# # print (history.history['loss'])
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()